# Семестровая работа. Вариант 1: Сортировка вставками
## Выполнил студент группы ФИТ-231 Сергиенко Александр Сергеевич

In [1]:
# Часть 1-3: Реализация алгоритма и теоретический анализ

import random
import time
import math

# 1. Стандартная сортировка вставками
def insertion_sort(some_list):
    """
    Сортировка вставками по возрастанию.
    Возвращает кортеж: (отсортированный список, количество сравнений, количество перестановок)
    """
    arr = some_list.copy()  # Работаем с копией, чтобы не изменять оригинал
    n = len(arr)
    comparisons = 0
    swaps = 0
    
    # Внешний цикл: проходим по всем элементам, начиная со второго
    for i in range(1, n):
        key = arr[i]
        j = i - 1
        
        # Внутренний цикл: сдвигаем элементы, которые больше key
        while j >= 0:
            comparisons += 1
            if arr[j] > key:
                arr[j + 1] = arr[j]
                swaps += 1
                j -= 1
            else:
                break
        
        # Вставляем key на правильное место
        arr[j + 1] = key
        if j + 1 != i:  # Если позиция изменилась
            swaps += 1  # Учитываем вставку как перестановку
    
    return arr, comparisons, swaps

# 2. Сортировка вставками с двоичным поиском
def binary_search(arr, key, low, high, comparisons_counter):
    """
    Бинарный поиск позиции для вставки.
    Возвращает индекс, куда нужно вставить key.
    """
    while low <= high:
        comparisons_counter[0] += 1
        mid = (low + high) // 2
        
        if key < arr[mid]:
            high = mid - 1
        else:
            low = mid + 1
    
    return low

def insertion_sort_with_bin_search(some_list):
    """
    Сортировка вставками с использованием двоичного поиска.
    Возвращает кортеж: (отсортированный список, количество сравнений, количество перестановок)
    """
    arr = some_list.copy()
    n = len(arr)
    comparisons = [0]  # Используем список для изменяемого значения
    swaps = 0
    
    for i in range(1, n):
        key = arr[i]
        
        # Находим позицию для вставки с помощью бинарного поиска
        pos = binary_search(arr, key, 0, i - 1, comparisons)
        
        # Сдвигаем элементы вправо
        for j in range(i, pos, -1):
            arr[j] = arr[j - 1]
            swaps += 1
        
        # Вставляем key на найденную позицию
        arr[pos] = key
        if pos != i:  # Если позиция изменилась
            swaps += 1  # Учитываем вставку как перестановку
    
    return arr, comparisons[0], swaps

# 3. Функции генерации тестовых данных
def generate_random(n):
    """a. Список случайных чисел"""
    return [random.randint(1, 10000) for _ in range(n)]

def generate_almost_reverse_sorted(n):
    """b. Почти отсортированный в обратную сторону список"""
    # Создаем список, отсортированный в обратном порядке
    arr = list(range(n, 0, -1))
    # Переставляем один элемент в середине
    mid = n // 2
    arr[mid], arr[mid + 1] = arr[mid + 1], arr[mid]
    return arr

def generate_half_pattern(n):
    """c. Половина данных равна 0, половина оставшихся 1, половина оставшихся 2 и т.д."""
    arr = []
    value = 0
    remaining = n
    
    while remaining > 0:
        count = remaining // 2 if remaining // 2 > 0 else remaining
        arr.extend([value] * count)
        remaining -= count
        value += 1
    
    # Перемешиваем, чтобы значения не были сгруппированы
    random.shuffle(arr)
    return arr

def generate_mostly_sorted(n):
    """d. Первые 95% списка упорядочены, последние 5% - случайные"""
    # Первые 95% отсортированы
    sorted_part = list(range(int(0.95 * n)))
    
    # Последние 5% случайные
    random_part = [random.randint(1, 10000) for _ in range(n - len(sorted_part))]
    
    return sorted_part + random_part

def generate_nearly_sorted(n):
    """e. Все элементы находятся в пределах 10 позиций от окончательного места"""
    # Создаем отсортированный список
    arr = list(range(n))
    
    # Немного перемешиваем, но так, чтобы элементы не уходили далеко от своих позиций
    for i in range(n):
        # Определяем диапазон, в котором можно перемещать элемент
        left = max(0, i - 10)
        right = min(n - 1, i + 10)
        
        # Случайно выбираем позицию для обмена
        j = random.randint(left, right)
        arr[i], arr[j] = arr[j], arr[i]
    
    return arr

# 4. Измерение времени выполнения
def measure_time(sort_func, data_generator, n_values, iterations=5):
    """
    Измеряет среднее время выполнения сортировки для разных размеров данных.
    
    Аргументы:
    sort_func -- функция сортировки
    data_generator -- функция генерации данных
    n_values -- список размеров данных
    iterations -- количество запусков для усреднения
    
    Возвращает словарь: {размер_данных: время_выполнения}
    """
    results = {}
    
    for n in n_values:
        total_time = 0
        
        for _ in range(iterations):
            data = data_generator(n)
            
            start_time = time.time()
            sort_func(data)
            end_time = time.time()
            
            total_time += (end_time - start_time)
        
        avg_time = total_time / iterations
        results[n] = avg_time
    
    return results

# 5. Вывод результатов в виде таблицы
def print_results_table(n_values, results_dict, title):
    """
    Выводит результаты в виде таблицы.
    
    Аргументы:
    n_values -- список размеров данных
    results_dict -- словарь {название_алгоритма_или_данных: {размер: время}}
    title -- заголовок таблицы
    """
    print("\n" + "=" * 80)
    print(title.center(80))
    print("=" * 80)
    
    # Заголовок таблицы
    header = f"{'Размер (n)':<12}"
    for name in results_dict.keys():
        header += f"{name:<20}"
    print(header)
    print("-" * (12 + 20 * len(results_dict)))
    
    # Данные таблицы
    for n in n_values:
        row = f"{n:<12}"
        for name, results in results_dict.items():
            if n in results:
                time_val = results[n]
                row += f"{time_val:.6f} сек".ljust(20)
            else:
                row += "N/A".ljust(20)
        print(row)

# 6. Демонстрация работы алгоритмов на примерах из задания
def demonstrate_sorting():
    """Демонстрирует работу алгоритмов на примерах из задания"""
    
    print("=" * 70)
    print("ПРИМЕР 1: Список [7, 3, 9, 4, 2, 5, 6, 1, 8]")
    print("=" * 70)
    
    test_list1 = [7, 3, 9, 4, 2, 5, 6, 1, 8]
    
    # Обычная сортировка вставками
    sorted1, comp1, swaps1 = insertion_sort(test_list1)
    print(f"Обычная сортировка вставками:")
    print(f"  Отсортированный список: {sorted1}")
    print(f"  Количество сравнений: {comp1}")
    print(f"  Количество перестановок: {swaps1}")
    print()
    
    # Сортировка вставками с двоичным поиском
    sorted1_bin, comp1_bin, swaps1_bin = insertion_sort_with_bin_search(test_list1)
    print(f"Сортировка вставками с двоичным поиском:")
    print(f"  Отсортированный список: {sorted1_bin}")
    print(f"  Количество сравнений: {comp1_bin}")
    print(f"  Количество перестановок: {swaps1_bin}")
    print()
    
    print("=" * 70)
    print("ПРИМЕР 2: Список [3, 5, 2, 9, 8, 1, 6, 4, 7]")
    print("=" * 70)
    
    test_list2 = [3, 5, 2, 9, 8, 1, 6, 4, 7]
    
    # Обычная сортировка вставками
    sorted2, comp2, swaps2 = insertion_sort(test_list2)
    print(f"Обычная сортировка вставками:")
    print(f"  Отсортированный список: {sorted2}")
    print(f"  Количество сравнений: {comp2}")
    print(f"  Количество перестановок: {swaps2}")
    print()
    
    # Сортировка вставками с двоичным поиском
    sorted2_bin, comp2_bin, swaps2_bin = insertion_sort_with_bin_search(test_list2)
    print(f"Сортировка вставками с двоичным поиском:")
    print(f"  Отсортированный список: {sorted2_bin}")
    print(f"  Количество сравнений: {comp2_bin}")
    print(f"  Количество перестановок: {swaps2_bin}")
    print()

# 7. Основная функция для проведения экспериментов
def run_experiments():
    """Проводит вычислительные эксперименты"""
    
    # Размеры данных для тестирования
    n_values = [100, 500, 1000, 2000, 4000, 6000, 8000, 10000]
    
    # Функции сортировки (только для измерения времени, без подсчета операций)
    def insertion_sort_time(data):
        result, _, _ = insertion_sort(data)
        return result
    
    def insertion_sort_bin_time(data):
        result, _, _ = insertion_sort_with_bin_search(data)
        return result
    
    # Типы данных
    data_generators = {
        'Случайные числа': generate_random,
        'Почти обратно отсорт.': generate_almost_reverse_sorted,
        'Половинный паттерн': generate_half_pattern,
        'В основном отсорт.': generate_mostly_sorted,
        'Почти отсорт.': generate_nearly_sorted
    }
    
    # Результаты для обычной сортировки по типам данных
    print("\n" + "=" * 80)
    print("РЕЗУЛЬТАТЫ ДЛЯ ОБЫЧНОЙ СОРТИРОВКИ ВСТАВКАМИ".center(80))
    print("=" * 80)
    
    std_results_by_data = {}
    for data_name, generator in data_generators.items():
        print(f"\nИзмерение для: {data_name}")
        results = measure_time(insertion_sort_time, generator, n_values, iterations=3)
        std_results_by_data[data_name] = results
        
        # Выводим результаты для этого типа данных
        print(f"{'Размер':<10} {'Время (сек)':<15}")
        print("-" * 25)
        for n in n_values:
            if n in results:
                print(f"{n:<10} {results[n]:<15.6f}")
    
    # Результаты для сортировки с двоичным поиском по типам данных
    print("\n" + "=" * 80)
    print("РЕЗУЛЬТАТЫ ДЛЯ СОРТИРОВКИ С ДВОИЧНЫМ ПОИСКОМ".center(80))
    print("=" * 80)
    
    bin_results_by_data = {}
    for data_name, generator in data_generators.items():
        print(f"\nИзмерение для: {data_name}")
        results = measure_time(insertion_sort_bin_time, generator, n_values, iterations=3)
        bin_results_by_data[data_name] = results
        
        # Выводим результаты для этого типа данных
        print(f"{'Размер':<10} {'Время (сек)':<15}")
        print("-" * 25)
        for n in n_values:
            if n in results:
                print(f"{n:<10} {results[n]:<15.6f}")
    
    # Сводная таблица сравнения для каждого типа данных при n=10000
    print("\n" + "=" * 80)
    print("СРАВНЕНИЕ ПРОИЗВОДИТЕЛЬНОСТИ ПРИ n=10000".center(80))
    print("=" * 80)
    print(f"{'Тип данных':<25} {'Обычная (сек)':<15} {'С поиском (сек)':<15} {'Ускорение':<15}")
    print("-" * 70)
    
    for data_name in data_generators.keys():
        if data_name in std_results_by_data and data_name in bin_results_by_data:
            std_time = std_results_by_data[data_name][10000]
            bin_time = bin_results_by_data[data_name][10000]
            speedup = std_time / bin_time if bin_time > 0 else 0
            
            print(f"{data_name:<25} {std_time:<15.6f} {bin_time:<15.6f} {speedup:<15.2f}")
    
    return std_results_by_data, bin_results_by_data

# 8. Анализ асимптотической сложности
def analyze_complexity():
    """Анализ асимптотической сложности алгоритмов"""
    
    print("=" * 80)
    print("АНАЛИЗ АСИМПТОТИЧЕСКОЙ СЛОЖНОСТИ".center(80))
    print("=" * 80)
    print()
    
    print("1. Обычная сортировка вставками:")
    print("   - Временная сложность:")
    print("     * Лучший случай (отсортированный массив): O(n)")
    print("     * Средний случай: O(n²)")
    print("     * Худший случай (обратно отсортированный массив): O(n²)")
    print("   - Пространственная сложность: O(1) (in-place алгоритм)")
    print()
    
    print("2. Сортировка вставками с двоичным поиском:")
    print("   - Временная сложность:")
    print("     * Поиск позиции: O(log n) вместо O(n) для каждого элемента")
    print("     * Но сдвиг элементов все равно требует O(n) операций")
    print("     * Общая сложность в худшем случае: O(n²) (из-за сдвигов)")
    print("     * Количество сравнений уменьшается до O(n log n)")
    print("   - Пространственная сложность: O(1) (in-place алгоритм)")
    print()
    
    print("=" * 80)
    print("ВЫВОДЫ ПО АНАЛИЗУ СЛОЖНОСТИ:".center(80))
    print("=" * 80)
    print("1. Оба алгоритма имеют квадратичную временную сложность в худшем случае.")
    print("2. Алгоритм с двоичным поиском уменьшает количество сравнений,")
    print("   но не улучшает асимптотическую сложность из-за операций сдвига.")
    print("3. На отсортированных данных оба алгоритма работают за линейное время O(n).")
    print("4. На обратно отсортированных данных оба алгоритма работают за квадратичное время O(n²).")

# 9. Подробное пошаговое выполнение для примеров из задания
def insertion_sort_detailed(arr, example_num):
    """Выводит подробное пошаговое выполнение сортировки"""
    print(f"\nДЕТАЛЬНОЕ ВЫПОЛНЕНИЕ ДЛЯ ПРИМЕРА {example_num}")
    print(f"Исходный список: {arr}")
    print()
    
    n = len(arr)
    comparisons = 0
    swaps = 0
    step = 1
    
    for i in range(1, n):
        key = arr[i]
        j = i - 1
        moved = False
        
        print(f"Шаг {step}: Вставляем элемент arr[{i}] = {key}")
        print(f"  Текущий список: {arr[:i]} | {key} | {arr[i+1:]}")
        
        # Внутренний цикл для поиска позиции
        while j >= 0:
            comparisons += 1
            print(f"    Сравниваем {arr[j]} > {key}? ", end="")
            
            if arr[j] > key:
                print(f"ДА -> сдвигаем {arr[j]} вправо")
                arr[j + 1] = arr[j]
                swaps += 1
                j -= 1
                moved = True
            else:
                print(f"НЕТ -> нашли позицию для вставки")
                break
        
        # Вставка ключа
        arr[j + 1] = key
        if moved:
            swaps += 1
        
        print(f"  Результат шага {step}: {arr}")
        print()
        step += 1
    
    print(f"ИТОГО для примера {example_num}:")
    print(f"  Сравнений: {comparisons}")
    print(f"  Перестановок: {swaps}")
    print(f"  Отсортированный список: {arr}")
    return arr

# 10. Запуск всех тестов и экспериментов
if __name__ == "__main__":
    print("=" * 80)
    print("СЕМЕСТРОВАЯ РАБОТА. ВАРИАНТ 1: СОРТИРОВКА ВСТАВКАМИ".center(80))
    print("=" * 80)
    print()
    
    # Примеры из задания с детальным выполнением
    print("\n" + "=" * 80)
    print("ЧАСТЬ 1-2: ПОДРОБНЫЙ АНАЛИЗ РАБОТЫ АЛГОРИТМА".center(80))
    print("=" * 80)
    
    # Пример 1
    arr1 = [7, 3, 9, 4, 2, 5, 6, 1, 8]
    insertion_sort_detailed(arr1.copy(), 1)
    
    # Пример 2  
    arr2 = [3, 5, 2, 9, 8, 1, 6, 4, 7]
    insertion_sort_detailed(arr2.copy(), 2)
    
    # Демонстрация работы алгоритмов с подсчетом операций
    print("\n" + "=" * 80)
    print("СРАВНЕНИЕ АЛГОРИТМОВ С ПОДСЧЕТОМ ОПЕРАЦИЙ".center(80))
    print("=" * 80)
    demonstrate_sorting()
    
    # Анализ сложности
    analyze_complexity()
    
    # Проведение экспериментов
    print("\n" + "=" * 80)
    print("ЧАСТЬ 4-5: ВЫЧИСЛИТЕЛЬНЫЕ ЭКСПЕРИМЕНТЫ".center(80))
    print("=" * 80)
    print("Проведение экспериментов... (это может занять некоторое время)")
    
    std_results, bin_results = run_experiments()
    
    # Итоговые выводы
    print("\n" + "=" * 80)
    print("ИТОГОВЫЕ ВЫВОДЫ ПО РЕЗУЛЬТАТАМ ЭКСПЕРИМЕНТОВ".center(80))
    print("=" * 80)
    
    print("\n1. На каких данных сортировка работает быстрее всего?")
    print("   - На почти отсортированных данных и данных, которые уже близки")
    print("     к отсортированному состоянию (типы 'Почти отсорт.' и 'В основном отсорт.').")
    
    print("\n2. На каких данных сортировка работает медленнее всего?")
    print("   - На обратно отсортированных данных или данных с паттерном,")
    print("     требующим много перестановок (типы 'Почти обратно отсорт.').")
    
    print("\n3. Есть ли отличие между обычной сортировкой и сортировкой с двоичным поиском?")
    print("   - Да, сортировка с двоичным поиском выполняет меньше сравнений,")
    print("     но это не всегда приводит к значительному ускорению из-за операций сдвига.")
    
    print("\n4. В каких случаях стоит использовать сортировку вставками?")
    print("   - На небольших массивах (до 1000 элементов)")
    print("   - На почти отсортированных данных")
    print("   - Как часть более сложных алгоритмов сортировки (например, в Timsort)")
    
    print("\n5. Рекомендации по выбору алгоритма:")
    print("   - Для маленьких или почти отсортированных массивов используйте обычную сортировку вставками")
    print("   - Для больших случайных массивов лучше использовать более эффективные алгоритмы")
    print("     (быстрая сортировка, сортировка слиянием и т.д.)")

              СЕМЕСТРОВАЯ РАБОТА. ВАРИАНТ 1: СОРТИРОВКА ВСТАВКАМИ               


                  ЧАСТЬ 1-2: ПОДРОБНЫЙ АНАЛИЗ РАБОТЫ АЛГОРИТМА                  

ДЕТАЛЬНОЕ ВЫПОЛНЕНИЕ ДЛЯ ПРИМЕРА 1
Исходный список: [7, 3, 9, 4, 2, 5, 6, 1, 8]

Шаг 1: Вставляем элемент arr[1] = 3
  Текущий список: [7] | 3 | [9, 4, 2, 5, 6, 1, 8]
    Сравниваем 7 > 3? ДА -> сдвигаем 7 вправо
  Результат шага 1: [3, 7, 9, 4, 2, 5, 6, 1, 8]

Шаг 2: Вставляем элемент arr[2] = 9
  Текущий список: [3, 7] | 9 | [4, 2, 5, 6, 1, 8]
    Сравниваем 7 > 9? НЕТ -> нашли позицию для вставки
  Результат шага 2: [3, 7, 9, 4, 2, 5, 6, 1, 8]

Шаг 3: Вставляем элемент arr[3] = 4
  Текущий список: [3, 7, 9] | 4 | [2, 5, 6, 1, 8]
    Сравниваем 9 > 4? ДА -> сдвигаем 9 вправо
    Сравниваем 7 > 4? ДА -> сдвигаем 7 вправо
    Сравниваем 3 > 4? НЕТ -> нашли позицию для вставки
  Результат шага 3: [3, 4, 7, 9, 2, 5, 6, 1, 8]

Шаг 4: Вставляем элемент arr[4] = 2
  Текущий список: [3, 4, 7, 9] | 2 | [5, 6, 1, 8]
    Сравниваем 

In [2]:
# Подробное пошаговое выполнение сортировки вставками для примера 1

def insertion_sort_step_by_step(arr):
    """Сортировка вставками с выводом состояния после каждой итерации"""
    print("Начальный список:", arr)
    print()
    
    n = len(arr)
    comparisons = 0
    swaps = 0
    
    for i in range(1, n):
        key = arr[i]
        j = i - 1
        
        print(f"Итерация {i}: обрабатываем элемент arr[{i}] = {key}")
        
        # Внутренний цикл для поиска позиции
        while j >= 0:
            comparisons += 1
            print(f"  Сравниваем {arr[j]} > {key}? ", end="")
            
            if arr[j] > key:
                print("Да -> сдвигаем arr[{}] вправо".format(j))
                arr[j + 1] = arr[j]
                swaps += 1
                j -= 1
            else:
                print("Нет -> останавливаемся")
                break
        
        # Вставка ключа
        arr[j + 1] = key
        if j + 1 != i:
            swaps += 1
            
        print(f"  Состояние после итерации {i}: {arr}")
        print()
    
    print(f"Итого: сравнений = {comparisons}, перестановок = {swaps}")
    return arr

# Пример 1 из задания
print("ПРИМЕР 1: [7, 3, 9, 4, 2, 5, 6, 1, 8]")
print("=" * 50)
arr1 = [7, 3, 9, 4, 2, 5, 6, 1, 8]
sorted1 = insertion_sort_step_by_step(arr1.copy())

ПРИМЕР 1: [7, 3, 9, 4, 2, 5, 6, 1, 8]
Начальный список: [7, 3, 9, 4, 2, 5, 6, 1, 8]

Итерация 1: обрабатываем элемент arr[1] = 3
  Сравниваем 7 > 3? Да -> сдвигаем arr[0] вправо
  Состояние после итерации 1: [3, 7, 9, 4, 2, 5, 6, 1, 8]

Итерация 2: обрабатываем элемент arr[2] = 9
  Сравниваем 7 > 9? Нет -> останавливаемся
  Состояние после итерации 2: [3, 7, 9, 4, 2, 5, 6, 1, 8]

Итерация 3: обрабатываем элемент arr[3] = 4
  Сравниваем 9 > 4? Да -> сдвигаем arr[2] вправо
  Сравниваем 7 > 4? Да -> сдвигаем arr[1] вправо
  Сравниваем 3 > 4? Нет -> останавливаемся
  Состояние после итерации 3: [3, 4, 7, 9, 2, 5, 6, 1, 8]

Итерация 4: обрабатываем элемент arr[4] = 2
  Сравниваем 9 > 2? Да -> сдвигаем arr[3] вправо
  Сравниваем 7 > 2? Да -> сдвигаем arr[2] вправо
  Сравниваем 4 > 2? Да -> сдвигаем arr[1] вправо
  Сравниваем 3 > 2? Да -> сдвигаем arr[0] вправо
  Состояние после итерации 4: [2, 3, 4, 7, 9, 5, 6, 1, 8]

Итерация 5: обрабатываем элемент arr[5] = 5
  Сравниваем 9 > 5? Да -> сдви

In [3]:
# Подробное пошаговое выполнение сортировки вставками для примера 2

# Пример 2 из задания
print("ПРИМЕР 2: [3, 5, 2, 9, 8, 1, 6, 4, 7]")
print("=" * 50)
arr2 = [3, 5, 2, 9, 8, 1, 6, 4, 7]
sorted2 = insertion_sort_step_by_step(arr2.copy())

ПРИМЕР 2: [3, 5, 2, 9, 8, 1, 6, 4, 7]
Начальный список: [3, 5, 2, 9, 8, 1, 6, 4, 7]

Итерация 1: обрабатываем элемент arr[1] = 5
  Сравниваем 3 > 5? Нет -> останавливаемся
  Состояние после итерации 1: [3, 5, 2, 9, 8, 1, 6, 4, 7]

Итерация 2: обрабатываем элемент arr[2] = 2
  Сравниваем 5 > 2? Да -> сдвигаем arr[1] вправо
  Сравниваем 3 > 2? Да -> сдвигаем arr[0] вправо
  Состояние после итерации 2: [2, 3, 5, 9, 8, 1, 6, 4, 7]

Итерация 3: обрабатываем элемент arr[3] = 9
  Сравниваем 5 > 9? Нет -> останавливаемся
  Состояние после итерации 3: [2, 3, 5, 9, 8, 1, 6, 4, 7]

Итерация 4: обрабатываем элемент arr[4] = 8
  Сравниваем 9 > 8? Да -> сдвигаем arr[3] вправо
  Сравниваем 5 > 8? Нет -> останавливаемся
  Состояние после итерации 4: [2, 3, 5, 8, 9, 1, 6, 4, 7]

Итерация 5: обрабатываем элемент arr[5] = 1
  Сравниваем 9 > 1? Да -> сдвигаем arr[4] вправо
  Сравниваем 8 > 1? Да -> сдвигаем arr[3] вправо
  Сравниваем 5 > 1? Да -> сдвигаем arr[2] вправо
  Сравниваем 3 > 1? Да -> сдвигаем a

In [4]:
# Сравнительный анализ количества операций

def compare_operations():
    """Сравнение количества операций для двух алгоритмов"""
    
    test_cases = {
        "Пример 1": [7, 3, 9, 4, 2, 5, 6, 1, 8],
        "Пример 2": [3, 5, 2, 9, 8, 1, 6, 4, 7]
    }
    
    print("СРАВНЕНИЕ КОЛИЧЕСТВА ОПЕРАЦИЙ")
    print("=" * 60)
    print(f"{'Тест':<15} {'Алгоритм':<25} {'Сравнения':<12} {'Перестановки':<12}")
    print("-" * 60)
    
    for test_name, test_data in test_cases.items():
        # Обычная сортировка
        _, comp_std, swaps_std = insertion_sort(test_data)
        
        # Сортировка с двоичным поиском
        _, comp_bin, swaps_bin = insertion_sort_with_bin_search(test_data)
        
        print(f"{test_name:<15} {'Обычная':<25} {comp_std:<12} {swaps_std:<12}")
        print(f"{test_name:<15} {'С бинарным поиском':<25} {comp_bin:<12} {swaps_bin:<12}")
        print("-" * 60)
    
    print("\nВЫВОДЫ:")
    print("1. Сортировка с двоичным поиском выполняет меньше сравнений")
    print("2. Количество перестановок в обоих алгоритмах примерно одинаково")
    print("3. На маленьких массивах выигрыш от бинарного поиска незначителен")

compare_operations()

СРАВНЕНИЕ КОЛИЧЕСТВА ОПЕРАЦИЙ
Тест            Алгоритм                  Сравнения    Перестановки
------------------------------------------------------------
Пример 1        Обычная                   24           26          
Пример 1        С бинарным поиском        19           26          
------------------------------------------------------------
Пример 2        Обычная                   22           22          
Пример 2        С бинарным поиском        18           22          
------------------------------------------------------------

ВЫВОДЫ:
1. Сортировка с двоичным поиском выполняет меньше сравнений
2. Количество перестановок в обоих алгоритмах примерно одинаково
3. На маленьких массивах выигрыш от бинарного поиска незначителен
